In [1]:
import pypyodbc
import pandas
import datetime
from datetime import date

# Define the Components of the Connection String.
server = '192.168.0.21'
port = '4900'
database = 'ICAM'
username = 'admin'
password = 'Clip_SERENA'

# Variables dates
date_today = datetime.datetime.today()
date_limit = date_today + datetime.timedelta(days=21)
formatted_date_limit = date_limit.strftime('%Y%m%d')
print(formatted_date_limit)

# Create a connection object.
connection_object: pypyodbc.Connection = pypyodbc.connect('DRIVER={HFSQL}; \
                        Server Name =' + server + '; \
                        Server Port=' + port + '; \
                        DATABASE=' + database + '; \
                        UID=' + username + '; \
                        PWD=' + password)
                        #Trusted_Connection=yes;

# Create a Cursor Object, using the connection.
cursor_object: pypyodbc.Cursor = connection_object.cursor()

# Define the Select Query.
sql_select = "SELECT GAMME.GACLEUNIK, \
                CHARGE.DAT as DATE_CHARGE, \
                CHARGRES.DAT as DATE_ORDO, \
                CHARGRES.HEUREDEB as HEURE_ORDO, \
                CLIENT.NOM, \
                AFFAIRE.NAF, \
                AFFAIRE.ETATAF, \
                GAMME.RANG, \
                GAMME.EN_PIECE, \
                GAMME.PHASE, \
                AFFAIRE.QTEAF, \
                GAMME.GA_PREP, \
                GAMME.GA_NBH, \
                GAMME.GA_NBHR, \
                AFFAIRE.TYPEAF, \
                GAMME.COFRAIS \
                FROM GAMME \
                LEFT JOIN AFFAIRE ON GAMME.NAF = AFFAIRE.NAF \
                LEFT JOIN ENGAM ON GAMME.ENCLEUNIK = ENGAM.ENCLEUNIK \
                LEFT JOIN CLIENT ON AFFAIRE.COCLI = CLIENT.COCLI \
                LEFT JOIN CHARGRES ON GAMME.GACLEUNIK = CHARGRES.GACLEUNIK \
                LEFT JOIN CHARGE ON GAMME.GACLEUNIK = CHARGE.GACLEUNIK \
                WHERE GAMME.COFRAIS like 'C35-2' \
                AND DATE_CHARGE <= '"+formatted_date_limit+"'"
                # AND EXISTS ( \
                # SELECT CHARGE.DAT \
                # FROM CHARGE \
                # LEFT JOIN GAMME GAMME2 ON CHARGE.GACLEUNIK = GAMME2.GACLEUNIK \
                # WHERE 1=1 \
                # AND GAMME2.GACLEUNIK = GAMME.GACLEUNIK \
                # AND CHARGE.DAT <= '"+formatted_date_limit+"' \
                # )
# AND AFFAIRE.ETATAF like 'F' \
# CHARGE.DAT as DATE_CHARGE,
# LEFT JOIN CHARGE ON GAMME.GACLEUNIK = CHARGE.GACLEUNIK
cursor_object.execute(sql_select)

# Define the column names.
#columns = [column[0] for column in cursor_object.statistics]
columns = [column[0] for column in cursor_object.description]

# Execute the Query.
# records = cursor_object.fetchmany(10)
records = cursor_object.fetchall()

print(records)

# Dump to a Pandas DataFrame.
ordo_df = pandas.DataFrame.from_records(
    data=records,
    columns=columns
)

# ordo_df

# print the head.
# print(ordo_df.head())

20240621
[(438231, datetime.date(2024, 5, 31), datetime.date(2024, 5, 30), datetime.time(14, 35), 'EWELLIX                                 ', 40211, 'F', '001/001                                                     ', 'TEMOIN FILETAGE 1168835       ', 30, 100.0, 2.0, 1.5, 0.0, '1', 'C35-2'), (438231, datetime.date(2024, 5, 30), datetime.date(2024, 5, 30), datetime.time(14, 35), 'EWELLIX                                 ', 40211, 'F', '001/001                                                     ', 'TEMOIN FILETAGE 1168835       ', 30, 100.0, 2.0, 1.5, 0.0, '1', 'C35-2'), (437557, datetime.date(2024, 6, 21), datetime.date(2024, 7, 18), datetime.time(15, 46), 'CRYOSTAR SAS                            ', 41120, 'F', '001                                                         ', '740234161-50                  ', 20, 2.0, 3.5, 3.0, 0.0, '1', 'C35-2'), (437557, datetime.date(2024, 6, 20), datetime.date(2024, 7, 18), datetime.time(15, 46), 'CRYOSTAR SAS                            ', 41120, 'F',

In [2]:
import json

# Supposons que vous ayez un DataFrame pandas appelé df (ici = donnees_affaires)
# Convertir le DataFrame en une chaîne JSON
json_data = ordo_df.to_json(orient='records')

# Charger la chaîne JSON en tant qu'objet JSON
data = json.loads(json_data)

print(json_data)

[{"gacleunik":438231,"date_charge":1717113600000,"date_ordo":1717027200000,"heure_ordo":"14:35:00","nom":"EWELLIX                                 ","naf":40211,"etataf":"F","rang":"001\/001                                                     ","en_piece":"TEMOIN FILETAGE 1168835       ","phase":30,"qteaf":100.0,"ga_prep":2.0,"ga_nbh":1.5,"ga_nbhr":0.0,"typeaf":"1","cofrais":"C35-2"},{"gacleunik":438231,"date_charge":1717027200000,"date_ordo":1717027200000,"heure_ordo":"14:35:00","nom":"EWELLIX                                 ","naf":40211,"etataf":"F","rang":"001\/001                                                     ","en_piece":"TEMOIN FILETAGE 1168835       ","phase":30,"qteaf":100.0,"ga_prep":2.0,"ga_nbh":1.5,"ga_nbhr":0.0,"typeaf":"1","cofrais":"C35-2"},{"gacleunik":437557,"date_charge":1718928000000,"date_ordo":1721260800000,"heure_ordo":"15:46:00","nom":"CRYOSTAR SAS                            ","naf":41120,"etataf":"F","rang":"001                                              

In [3]:
ordo_df

,gacleunik,date_charge,date_ordo,heure_ordo,nom,naf,etataf,rang,en_piece,phase,qteaf,ga_prep,ga_nbh,ga_nbhr,typeaf,cofrais
0,438231,2024-05-31,2024-05-30,14:35:00,EWELLIX,40211,F,001/001 ...,TEMOIN FILETAGE 1168835,30,100.0,2.0,1.50,0.0,1,C35-2
1,438231,2024-05-30,2024-05-30,14:35:00,EWELLIX,40211,F,001/001 ...,TEMOIN FILETAGE 1168835,30,100.0,2.0,1.50,0.0,1,C35-2
2,437557,2024-06-21,2024-07-18,15:46:00,CRYOSTAR SAS,41120,F,001 ...,740234161-50,20,2.0,3.5,3.00,0.0,1,C35-2
3,437557,2024-06-20,2024-07-18,15:46:00,CRYOSTAR SAS,41120,F,001 ...,740234161-50,20,2.0,3.5,3.00,0.0,1,C35-2
4,437557,2024-06-19,2024-07-18,15:46:00,CRYOSTAR SAS,41120,F,001 ...,740234161-50,20,2.0,3.5,3.00,0.0,1,C35-2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
153,430563,2024-06-03,2024-07-19,14:46:00,SNCF RESEAU,40641,F,001/003 ...,79420255 - REP 04,40,30.0,1.0,7.75,0.0,1,C35-2
154,430563,2024-05-31,2024-07-19,14:46:00,SNCF RESEAU,40641,F,001/003 ...,79420255 - REP 04,40,30.0,1.0,7.75,0.0,1,C35-2
155,430563,2024-05-30,2024-07-19,14:46:00,SNCF RESEAU,40641,F,001/003 ...,79420255 - REP 04,40,30.0,1.0,7.75,0.0,1,C35-2
156,431668,2024-05-31,2024-05-31,02:05:00,SNCF RESEAU,40792,F,001/003 ...,79420255 - REP 04,40,30.0,1.0,7.75,0.7,1,C35-2
